# Setup

In [1]:
import pandas as pd
import os
import codecs
from bs4 import BeautifulSoup 
import codecs
import spacy

# Extract

In [26]:
nlp = spacy.load("en_core_web_lg")

def xfm(raw_text):
    plain_text = BeautifulSoup(raw_text).text
    doc = nlp(plain_text)
    ents = [ent for ent in doc.ents]
    return (plain_text, ents)

In [4]:
files = [fn for fn in os.listdir("./resume_corpus") if fn[-4:] == ".lab" ]

In [31]:
recs = []
for fn in files[0:300]:

    fn_prefix = fn[:-4]
    with codecs.open("./resume_corpus/" + fn_prefix + ".lab", 'r', encoding='utf-8', errors='ignore') as f:
        labels = f.read()         
    labels_split = labels.split("\n")

    with codecs.open("./resume_corpus/" + fn_prefix + ".txt", 'r', encoding='utf-8', errors='ignore') as f:
        raw_text = f.read()         
    (plain_text, ents) = xfm(raw_text)
    
    recs.append({"id": fn_prefix, "labels": labels_split, "cv": plain_text, "entities": ents})
    

In [30]:
recs

[{'id': '16939',
  'labels': ['Network_Administrator'],
  'cv': "Network Administrator Network Administrator Network Administrator - United States Air Force Tucson, AZ Work Experience Network Administrator United States Air Force - Tucson, AZ June 2011 to Present Tucson, Arizona June 2011 - Present Network Administrator As a Staff Sergeant, lead the daily activities of 10 personnel responsible for the maintenance of classified/unclassified telecommunications and networking equipment; enforce adherence to strict quality standards in order to continuously meet customer requirements and surpass deadlines/performance benchmarks.  Deploy, sustain, troubleshoot and repair standard network infrastructure systems, IP detection systems and General Dynamics encryption devices.  Support unit operations by organizing office procedures, maintaining workplace systems, supervising training efforts and managing personnel; plan staff schedules, collect and assign workload and develop and implement metr